In [6]:
from pyspark import SparkContext, SparkConf
from pyspark.mllib.feature import HashingTF, IDF

In [15]:
filepath = "../../datasets/MLCourse/subset-small.tsv"
# with open(filepath) as f:
#     print(f.readline())
#     print(f.readline())
pd.read_csv(filepath, sep="\t", header=None).head()

,0,1,2,3
0,12,Anarchism,2008-12-30 06:23:05,Anarchism (sometimes referred to as libertaria...
1,25,Autism,2008-12-24 20:41:05,Autism is a brain development disorder that is...
2,39,Albedo,2008-12-29 18:19:09,The albedo of an object is the extent to which...
3,290,A,2008-12-27 04:33:16,The letter A is the first letter in the Latin ...
4,303,Alabama,2008-12-29 08:15:47,"Alabama (formally, the State of Alabama; ) is ..."


In [3]:
conf = SparkConf().setMaster("local").setAppName("SparkTFIDF")
sc = SparkContext(conf=conf)

In [18]:
raw_data = sc.textFile(filepath)
fields = raw_data.map(lambda x: x.split("\t"))
documents = fields.map(lambda x: x[3].split(" "))
document_names = fields.map(lambda x: x[1])

In [20]:
hashingTF = HashingTF(100000) #100K hash buckets
tf = hashingTF.transform(documents)

In [21]:
tf.cache()
idf = IDF(minDocFreq=2).fit(tf)
tfidf = idf.transform(tf)

In [36]:
gettysburgTF = hashingTF.transform(["Gettysburg"])
gettysburg_hash_value = gettysburgTF.indices[0]

In [37]:
gettysburg_relevance = tfidf.map(lambda x: x[int(gettysburg_hash_value)])

In [40]:
zipped_results = gettysburg_relevance.zip(document_names)
print("Best document for Gettysburg:")
print(zipped_results.max())

Best document for Gettysburg:
(5.52246041819533, 'Auguste Rodin')
